In [1]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 960.0 kB/s eta 0:00:12
   - -------------------------------------- 0.3/10.6 MB 5.0 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/10.6 MB 8.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/10.6 MB 10.8 MB/s eta 0:00:01
   ----------- ---------------------------- 3.0/10.6 MB 13.8 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.6 MB 14.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.3/10.6 MB 14.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.6/10.6 MB 12.7 MB/s eta 0:00:01
   ------------------ --------------------- 4.9/10.6 MB 12.0 MB/s eta 0:00:01
   ------------------- -------------------- 5.1/10.6 MB 11.3 MB/s eta 0:00:01
   -------------------- ------------------- 5.3/10.6 MB 10.3 MB/s eta 0:0

In [ ]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Coin Collector")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# Clock
clock = pygame.time.Clock()
FPS = 60

# Load background images
game_background_img = pygame.image.load("C:/Users/Rehan/OneDrive/Desktop/PPS Project/background.png").convert()
game_background_img = pygame.transform.scale(game_background_img, (WIDTH, HEIGHT))

start_background_img = pygame.image.load("C:/Users/Rehan/OneDrive/Desktop/PPS Project/start_screen.png").convert()
start_background_img = pygame.transform.scale(start_background_img, (WIDTH, HEIGHT))

# Load coin sprite sheet
coin_sheet = pygame.image.load("C:/Users/Rehan/OneDrive/Desktop/PPS Project/coin.png").convert_alpha()
COIN_FRAME_WIDTH = 20
COIN_FRAME_HEIGHT = 20

COIN_COLUMNS = coin_sheet.get_width() // COIN_FRAME_WIDTH
print("COIN_COLUMNS:", COIN_COLUMNS)

coin_frames = []
for i in range(COIN_COLUMNS):
    frame = coin_sheet.subsurface(pygame.Rect(i * COIN_FRAME_WIDTH, 0, COIN_FRAME_WIDTH, COIN_FRAME_HEIGHT))
    frame = pygame.transform.scale(frame, (40, 40))
    coin_frames.append(frame)

coin_index = 0
coin_anim_speed = 0.2

coin_rect = coin_frames[0].get_rect(center=(random.randint(40, WIDTH - 40),
                                            random.randint(40, HEIGHT - 40)))

# Load player sprite sheet 
player_sheet = pygame.image.load("C:/Users/Rehan/OneDrive/Desktop/PPS Project/player.png").convert_alpha()
PLAYER_FRAME_WIDTH = 32
PLAYER_FRAME_HEIGHT = 32

PLAYER_COLUMNS = player_sheet.get_width() // PLAYER_FRAME_WIDTH
print("PLAYER_COLUMNS:", PLAYER_COLUMNS)

player_frames = {'down': [], 'left': [], 'right': [], 'up': []}
directions = ['down', 'left', 'right', 'up']

for row, direction in enumerate(directions):
    for col in range(PLAYER_COLUMNS):
        frame = player_sheet.subsurface(pygame.Rect(
            col * PLAYER_FRAME_WIDTH,
            row * PLAYER_FRAME_HEIGHT,
            PLAYER_FRAME_WIDTH,
            PLAYER_FRAME_HEIGHT
        ))
        frame = pygame.transform.scale(frame, (90, 90))
        player_frames[direction].append(frame)

player_rect = player_frames['down'][0].get_rect(center=(WIDTH // 2, HEIGHT // 2))
player_index = 0
player_anim_speed = 10  # For delta_time compatibility
player_speed = 3

# Load font
font = pygame.font.Font("C:/Users/Rehan/OneDrive/Desktop/PPS Project/Minecraft.ttf", 26)
big_font = pygame.font.Font("C:/Users/Rehan/OneDrive/Desktop/PPS Project/Minecraft.ttf", 48)

# Load sound
pygame.mixer.init()
coin_sound = pygame.mixer.Sound("C:/Users/Rehan/OneDrive/Desktop/PPS Project/coin_sound.wav")

# Score
score = 0
time_limit = 30  # Game duration in seconds

# Start screen function
def show_start_screen():
    button_rect = pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2 + 50, 200, 80)
    while True:
        screen.blit(start_background_img, (0, 0))

        title_text = big_font.render("Coin Collector", True, WHITE)
        screen.blit(title_text, (WIDTH // 2 - title_text.get_width() // 2, HEIGHT // 3))

        pygame.draw.rect(screen, WHITE, button_rect, border_radius=12)
        start_text = font.render("Start Game", True, BLACK)
        screen.blit(start_text, (button_rect.x + button_rect.width // 2 - start_text.get_width() // 2,
                                 button_rect.y + button_rect.height // 2 - start_text.get_height() // 2))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if button_rect.collidepoint(event.pos):
                    return

        pygame.display.flip()
        clock.tick(FPS)

# Game over screen with Restart and Menu
def get_game_over_surface(final_score):
    surface = pygame.Surface((WIDTH, HEIGHT))
    surface.blit(start_background_img, (0, 0))

    over_text = big_font.render("Game Over", True, WHITE)
    score_text = font.render(f"Final Score: {final_score}", True, WHITE)

    surface.blit(over_text, (WIDTH // 2 - over_text.get_width() // 2, HEIGHT // 3))
    surface.blit(score_text, (WIDTH // 2 - score_text.get_width() // 2, HEIGHT // 3 + 80))

    restart_rect = pygame.Rect(WIDTH // 2 - 220, HEIGHT // 2 + 50, 180, 60)
    menu_rect = pygame.Rect(WIDTH // 2 + 40, HEIGHT // 2 + 50, 180, 60)

    pygame.draw.rect(surface, WHITE, restart_rect, border_radius=12)
    restart_text = font.render("Restart", True, BLACK)
    surface.blit(restart_text, (restart_rect.x + restart_rect.width // 2 - restart_text.get_width() // 2,
                                restart_rect.y + restart_rect.height // 2 - restart_text.get_height() // 2))

    pygame.draw.rect(surface, WHITE, menu_rect, border_radius=12)
    menu_text = font.render("Menu", True, BLACK)
    surface.blit(menu_text, (menu_rect.x + menu_rect.width // 2 - menu_text.get_width() // 2,
                             menu_rect.y + menu_rect.height // 2 - menu_text.get_height() // 2))

    return surface, restart_rect, menu_rect

# Crossfade transition function
def crossfade_to_surface(current_surface, target_surface, duration=800):
    fade_overlay = target_surface.copy()
    clock_ = pygame.time.Clock()
    start_ticks = pygame.time.get_ticks()
    while True:
        elapsed = pygame.time.get_ticks() - start_ticks
        alpha = min(255, int((elapsed / duration) * 255))
        fade_overlay.set_alpha(alpha)
        screen.blit(current_surface, (0, 0))
        screen.blit(fade_overlay, (0, 0))
        pygame.display.update()
        clock_.tick(FPS)
        if alpha >= 255:
            break

# Start screen first
show_start_screen()

# Main game loop
def start_game():
    global score, player_rect, coin_rect
    score = 0
    player_rect.center = (WIDTH // 2, HEIGHT // 2)
    coin_rect.center = (random.randint(40, WIDTH - 40), random.randint(40, HEIGHT - 40))
    start_time = pygame.time.get_ticks()
    player_direction = 'down'

    running = True
    while running:
        delta_time = clock.get_time() / 1000
        elapsed_time = (pygame.time.get_ticks() - start_time) / 1000
        remaining_time = max(0, int(time_limit - elapsed_time))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        keys = pygame.key.get_pressed()
        moving = False

        if keys[pygame.K_LEFT]:
            player_rect.x -= player_speed
            player_direction = 'left'
            moving = True
        if keys[pygame.K_RIGHT]:
            player_rect.x += player_speed
            player_direction = 'right'
            moving = True
        if keys[pygame.K_UP]:
            player_rect.y -= player_speed
            player_direction = 'up'
            moving = True
        if keys[pygame.K_DOWN]:
            player_rect.y += player_speed
            player_direction = 'down'
            moving = True

        player_rect.x = max(0, min(player_rect.x, WIDTH - player_rect.width))
        player_rect.y = max(0, min(player_rect.y, HEIGHT - player_rect.height))

        global player_index, coin_index
        if moving:
            player_index += player_anim_speed * delta_time
            if player_index >= PLAYER_COLUMNS:
                player_index = 0
        else:
            player_index = 0

        current_player_frame = player_frames[player_direction][int(player_index)]

        coin_index += coin_anim_speed * delta_time * FPS
        if coin_index >= COIN_COLUMNS:
            coin_index = 0

        current_coin_frame = coin_frames[int(coin_index)]

        if player_rect.colliderect(coin_rect):
            score += 1
            coin_rect.center = (random.randint(40, WIDTH - 40), random.randint(40, HEIGHT - 40))
            coin_sound.play()

        screen.blit(game_background_img, (0, 0))
        screen.blit(current_player_frame, player_rect)
        screen.blit(current_coin_frame, coin_rect)

        score_text = font.render(f"Score: {score}", True, WHITE)
        screen.blit(score_text, (10, 10))

        timer_text = font.render(f"Time Left: {remaining_time}s", True, WHITE)
        screen.blit(timer_text, (WIDTH - timer_text.get_width() - 10, 10))

        pygame.display.flip()
        clock.tick(FPS)

        if remaining_time == 0:
            current_screen = screen.copy()
            game_over_surface, restart_rect, menu_rect = get_game_over_surface(score)
            crossfade_to_surface(current_screen, game_over_surface)

            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if restart_rect.collidepoint(event.pos):
                            start_game()
                            return
                        elif menu_rect.collidepoint(event.pos):
                            show_start_screen()
                            start_game()
                            return

                screen.blit(game_over_surface, (0, 0))
                pygame.display.flip()
                clock.tick(FPS)

# Start the first game
start_game()
